In [91]:
import requests
import json
from pathlib import Path
import os
import numpy as np
import pickle
from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.display import clear_output, display
import pandas as pd
from random import randrange
from hazm import *

In [92]:
sparqlEndpoint = "http://localhost:8890//sparql"
sparql = SPARQLWrapper(sparqlEndpoint)

In [93]:
def fixURI(uri):
    splited = uri.split('/')
    if('relation' in splited[len(splited)-1]):
        fixedURI = ''
        for i in range (0, len(splited)-2):
            fixedURI += splited[i] + '/'
        fixedURI += splited[len(splited)-2]
        return fixedURI
    else:
        return uri

In [94]:
def getNameFromURI(uri):
    splited = fixURI(uri).split('/')
    splitedEntityName = splited[len(splited)-1].split('_')
    entityName = ''
    for i in range (0, len(splitedEntityName)-1):
        entityName += splitedEntityName[i] + ' '
    entityName += splitedEntityName[len(splitedEntityName)-1]
    return entityName
    

In [95]:
class complexSentenceTemplate:
    sType = ''
    sHead = []
    usedSHead = []
    sR1 = ''
    sE1 = ''
    sE1URI = ''
    sE2 = ''
    sE2URI = ''
    sE3 = ''
    sE3URI = ''
    sDC = []
    usedSDC = []
    sMid = []
    usedSMid = []
    sTail = []
    usedSTail = []
    sRelation = ''
    isReverse = 0
    
    def __init__(self, st, shead, sr1, se1, se1URI, se2, se2URI, se3, se3URI, sdc, smid, stail, sr, isRev):
        self.sType = st
        self.sHead = shead
        self.usedSHead = []
        if(len(self.sHead) == 0):
            self.sHead.append("*")
        self.sR1 = sr1
        self.sE1 = se1
        self.sE1URI = se1URI
        self.sE2 = se2
        self.sE2URI = se2URI
        self.sE3 = se3
        self.sE3URI = se3URI
        self.sDC = sdc
        self.usedSDC = []
        if(len(self.sDC) == 0):
            self.sDC.append("*")
        self.sMid = smid
        self.usedSMid = []
        if(len(self.sMid) == 0):
            self.sMid.append(" ")
        self.sTail = stail
        self.usedSTail = []
        if(len(self.sTail) == 0):
            self.sTail.append("*")
        self.sRelation = sr
        self.isReverse = isRev
        
    def __init__(self, st, shead, sr1, se1, se2, se3, sdc, smid, stail, sr, isRev):
        self.sType = st
        self.sHead = shead
        self.usedSHead = []
        if(len(self.sHead) == 0):
            self.sHead.append("*")
        self.sR1 = sr1
        self.sE1 = se1
        self.sE2 = se2
        self.sE3 = se3
        self.sDC = sdc
        self.usedSDC = []
        if(len(self.sDC) == 0):
            self.sDC.append("*")
        self.sMid = smid
        self.usedSMid = []
        if(len(self.sMid) == 0):
            self.sMid.append("*")
        self.sTail = stail
        self.usedSTail = []
        if(len(self.sTail) == 0):
            self.sTail.append("*")
        self.sRelation = sr
        self.isReverse = isRev

In [96]:
def makeSentence(st):
    print(len(st.sHead))
    print(len(st.usedSHead) >= len(st.sHead))
    if (len(st.usedSHead) >= len(st.sHead)):
        return None
    headIndex = len(st.usedSHead)
    if (len(st.usedSDC) >= len(st.sDC)):
        st.usedSDC = []
        st.usedSHead.append(st.sHead[len(st.usedSHead)])
    dcIndex = len(st.usedSDC)
    if (len(st.usedSMid) >= len(st.sMid)):
        st.usedSMid = []
        st.usedSDC.append(st.sDC[len(st.usedSDC)])
    midIndex = len(st.usedSMid)
    if (len(st.usedSTail) >= len(st.sTail)):
        st.usedSTail = []
        st.usedSMid.append(st.sMid[len(st.usedSMid)])
    tailIndex = len(st.usedSTail)
    st.usedSTail.append(st.sTail[len(st.usedSTail)])
    sentence = str(st.sHead[headIndex]) + " " + str(st.sE1) + " " + str(st.sDC[dcIndex]) + " " + str(st.sE2) + " " + str(st.sMid[midIndex]) + " " + str(st.sE3) + " " + str(st.sTail[tailIndex])
    sentence = sentence.replace('nan', '').replace('  ', ' ').replace('  ', ' ').replace(' ، ', '، ').replace('*', '')
    if(sentence[0] == ' '):
        sentence = sentence[1:]
    if(sentence[len(sentence) - 1] == ' '):
        sentence = sentence[0: len(sentence) - 1]
    print(sentence)
    return sentence

In [97]:
print(makeSentence(templates[4]))

1
True
None


In [98]:
# -*- coding: utf-8 -*-
def isEnglishChar(s):
    if(s in "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"):
        return True
    else:
        return False
    
def isEnglish(s):
    hasEn = False
    for i in range (0, len(s)):
        if(isEnglishChar(s[i])):
            hasEn = True
    return hasEn

def isURIValid(uri):
    if("http" in uri):
        return True
    else:
        return False

dfs = pd.read_excel("complexSentenceTemplates.xlsx", sheet_name='Sheet1')
templates = []
for i in range (0, len(dfs)):
    st = complexSentenceTemplate(dfs['Type'][i], str(dfs['Head'][i]).split(','), dfs['R1'][i], dfs['Fisrt Entity'][i], dfs['Second Entity'][i], dfs['Third Entity'][i], str(dfs['Dependent Clause'][i]).split(','), str(dfs['Mid'][i]).split(','), str(dfs['Tail'][i]).split(','), dfs['Relation'][i], dfs['isReverse'][i])
    templates.append(st)
    
dfss = pd.read_excel("Properties_refined_two_side_entity_filterd.xlsx", sheet_name=None)
relations = []
for i in range (0, len(dfss)):
    relations.append(dfss['URL'][i])
print((str(dfss['translatedLabel'][relations.index(relations[0])])))
entities = {}
generatedRelations = []
for i in range (0, len(templates)):
# for i in range (1, 2):
    if(templates[i].isReverse == 0):
        stringQuery = """SELECT ?s ?o1 ?o2 WHERE{
        ?s <""" + str(templates[i].sR1) + """> ?o1
        .
        ?s <""" + str(templates[i].sRelation) + """> ?o2
        }"""
    else:
        stringQuery = """SELECT ?s ?o1 ?o2 WHERE{
        ?o1 <""" + str(templates[i].sR1) + """> ?s
        .
        ?s <""" + str(templates[i].sRelation) + """> ?o2
        }"""
    print(stringQuery)
    sparql.setQuery(stringQuery)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    entities = {}
    for j in range((9*len(results['results']['bindings']))//10, 10*len(results['results']['bindings'])//10):
#         if(templates[i].isReverse == 0):
        subjectURI = results['results']['bindings'][j]['s']['value']
#         print(subjectURI)
        object1URI = results['results']['bindings'][j]['o1']['value']
#         print(object1URI)
#         else:
#             subjectURI = results['results']['bindings'][j]['o1']['value']
#             object1URI = results['results']['bindings'][j]['s']['value']
        object2URI = results['results']['bindings'][j]['o2']['value']
#         print(object2URI)
        
        if(isURIValid(subjectURI) and isURIValid(object1URI) and isURIValid(object2URI)):
            pass
        else:
            continue
        if(isEnglish(getNameFromURI(subjectURI))):
            continue
        templates[i].sE1 = getNameFromURI(subjectURI)
        templates[i].sE1RUI = fixURI(subjectURI)
        if(isEnglish(getNameFromURI(object1URI))):
            continue
        templates[i].sE2 = getNameFromURI(object1URI)
        templates[i].sE2RUI = fixURI(object1URI)
        if(isEnglish(getNameFromURI(object2URI))):
            continue
        templates[i].sE3 = getNameFromURI(object2URI)
        templates[i].sE3RUI = fixURI(object2URI)
        if(getNameFromURI(subjectURI) in entities):
            continue
        else:
            entities[getNameFromURI(subjectURI)] = getNameFromURI(object1URI)
            
        newRelation = {}
        newRelation['sentence'] = makeSentence(templates[i])
        if(not(newRelation['sentence'] == None)):
            head = {}
            head['word'] = templates[i].sE1
            head['start'] = newRelation['sentence'].find(head['word'])
            head['length'] = len(head['word'])
            head['id'] = templates[i].sE1RUI
            newRelation['head'] = head
            tail = {}
            tail['word'] = templates[i].sE2
            tail['start'] = newRelation['sentence'].find(tail['word'])
            tail['length'] = len(tail['word'])
            tail['id'] = templates[i].sE2RUI
            newRelation['tail'] = tail
            relation = {}
    #         print(templates[i].sR1)
            relation['word'] = str(dfss['translatedLabel'][relations.index(templates[i].sR1)])
            relation['id'] = templates[i].sR1
            newRelation['relation'] = relation
            newRelation['why'] = 'generated'

            if(not(newRelation in generatedRelations)):
                generatedRelations.append(newRelation)
    #         print(newRelation)

            newRelation2 = {}
            newRelation2['sentence'] = newRelation['sentence']
            head = {}
            head['word'] = templates[i].sE1
            head['start'] = newRelation2['sentence'].find(head['word'])
            head['length'] = len(head['word'])
            head['id'] = templates[i].sE1RUI
            newRelation2['head'] = head
            tail = {}
            tail['word'] = templates[i].sE3
            tail['start'] = newRelation2['sentence'].find(tail['word'])
            tail['length'] = len(tail['word'])
            tail['id'] = templates[i].sE3RUI
            newRelation2['tail'] = tail
            relation = {}
    #         print(templates[i].sR1)
            relation['word'] = str(dfss['translatedLabel'][relations.index(templates[i].sRelation)])
            relation['id'] = templates[i].sRelation
            newRelation2['relation'] = relation
            newRelation2['why'] = 'generated'

            if(not(newRelation2 in generatedRelations)):
                generatedRelations.append(newRelation2)
#         print(newRelation2)
#         print(subjectURI)
#         print(object1URI)
#         print(object2URI)


شهرستان
SELECT ?s ?o1 ?o2 WHERE{
        ?s <http://fkg.iust.ac.ir/ontology/birthPlace> ?o1
        .
        ?s <http://fkg.iust.ac.ir/ontology/deathPlace> ?o2
        }
1
False
سامرست موآم متولد پاریس در فرانسه وفات کرد
1
False
ساموئل بی. مکسی متولد کنتاکی در آرکانزاس فوت کرد
1
False
ساموئل دبلیو. دانا متولد کنتیکت در میدل‌تاون، کنتیکت درگذشت
1
False
ساموئل دو متولد تازون در مونروویا به سفر آخرت شتافت
1
False
ساموئل سی. پومروی متولد ساتهمپتون، ماساچوست در وایتینزویل، ماساچوست دعوت حق را لبیک گفت
1
False
ساموئل ماد متولد مریلند در مریلند غزل خداحافظی را خواند
1
False
ساموئل مارشاک متولد وورونژ در مسکو به ابدیت پیوست
1
False
ساموئل پیرپونت لانگلی متولد روکسبوری، بوستون در ایکن، کارولینای جنوبی از دنیا رفت
1
False
ساموئل کینگ آلیسون متولد شیکاگو در شیکاگو جان باخت
1
False
سامون سامدیکون متولد مگاتن در جاکارتا جان به جان‌آفرین تسلیم کرد
1
False
سانتوس ایریاته متولد کنلونس، اروگوئه در مونته‌ویدئو چشم از جهان فروبست
1
False
ساندرو سالوادور متولد ایتالیا در ایتالیا چشم از جهان بست
1
False
س

1
False
از آثار خوآن لاپورتا متولد کاتالونیا می‌توان به باشگاه فوتبال بارسلونا اشاره کرد
1
False
از آثار کاترین کارتلیج متولد انگلستان می‌توان به برهنه (فیلم) اشاره کرد
1
False
از آثار کانن اوبراین متولد بروکلاین، ماساچوست می‌توان به سیمپسون‌ها اشاره کرد
1
False
از آثار نگار بوبان زاده‌ی ایران می‌توان به به‌هنگام اشاره کرد
1
False
از آثار منصور خورشیدی به دنیا آمده‌ی بهشهر می‌توان به آبی ناگهان اشاره کرد
1
False
از آثار ناصر وحدتی اهل ایران می‌توان به آها بوگو(آلبوم موسیقی) اشاره کرد
1
False
از آثار ناصرالحق اطروش متولد مدینه می‌توان به تفسیر کبیر اشاره کرد
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
SELECT ?s ?o1 ?o2 WHERE{
        ?s <http://fkg.iust.ac.ir/ontology/nationality> ?o1
        .
        ?s <http://

1
False
لوک موله فیلم‌نامه‌نویس در پاریس به دنیا آمد
SELECT ?s ?o1 ?o2 WHERE{
        ?o1 <http://fkg.iust.ac.ir/ontology/musicComposer> ?s
        .
        ?s <http://fkg.iust.ac.ir/ontology/birthPlace> ?o2
        }
1
False
لئو فوربستاین موسیقیدان سن کوئنتین (فیلم ۱۹۳۷) در ایالات متحده آمریکا به دنیا آمد
1
False
جورجو گازلینی موسیقیدان قرمز تیره در میلان زاده شد
1
False
یوری شاپورین موسیقیدان مینین و پوژارسکی (فیلم) در امپراتوری روسیه متولد شد
1
False
الی وروبل موسیقیدان پیاده‌روی لاس‌زنی در ایالات متحده آمریکا به دنیا آمد
1
False
کریگ سافان موسیقیدان بیگانه (فیلم ۱۹۸۷) در لس آنجلس زاده شد
1
False
ارنست گلد (آهنگساز) موسیقیدان ستیزه‌جویان در اتریش متولد شد
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
T

1
False
پردیس بین‌الملل کیش دانشگاه صنعتی شریف در جزیره کیش در کشور ایران قرار گرفته است
1
False
تئاتر ملی سارایوو در سارایوو در کشور بوسنی و هرزگوین واقع شده است
1
False
تئاتر ملی ملبورن در ملبورن در کشور استرالیا قرار دارد
1
False
تئاتر ملی کرواسی در اسپلیت در اسپلیت در کشور کرواسی است
1
False
تئاتر ملی کرواسی در زاگرب در زاگرب در کشور کرواسی قرار گرفته است
1
False
تئاتر و سالن کنسرت ملی تایپه در تایپه تایوان واقع شده است
1
False
پلی‌تکنیک مونترآل در مونترآل کانادا قرار دارد
SELECT ?s ?o1 ?o2 WHERE{
        ?o1 <http://fkg.iust.ac.ir/ontology/editing> ?s
        .
        ?s <http://fkg.iust.ac.ir/ontology/birthPlace> ?o2
        }
1
False
ویلیام اچ. زیگلر تدوین‌گر نخاله‌ها در دریا متولد فیلادلفیا، پنسیلوانیا است
1
False
ویلیام هلمز تدوین‌گر هوانورد (فیلم ۱۹۲۹) متولد ایالات متحده آمریکا است
1
False
ویلیام همیلتون (تدوین‌گر) تدوین‌گر گوژپشت نتردام (فیلم ۱۹۳۹) زاده‌ی پنسیلوانیا است
1
False
ویلیام هورنبک تدوین‌گر آمریکایی آرام (فیلم ۱۹۵۸) به دنیا آمده‌ی لس‌آنجلس است
1
False
ژان-مارک ولی

3
False
ترک خوارگردان توسط پیکسیز خوانده و الکترا رکوردز آن را منتشر کرده است
3
False
ترک خودت را آنطور که هستی نشان بده توسط نیروانا (گروه موسیقی) خوانده و گفن رکوردز منتشر کرده
3
False
ترک خودتو غرق کن توسط امینم خوانده و شیدی رکوردز منتشر کرده است
3
False
ترک خودکشی درد ندارد توسط جانی مندل خوانده و کلمبیا رکوردز آن را منتشر کرده است
3
False
ترک پادشاه کوه (ترانه) توسط کیت بوش خوانده و شرکت ایمی منتشر کرده
3
False
ترک پارانوئید (ترانه) توسط بلک سبث خوانده و شرکت ورتیگو رکوردز منتشر کرده است
3
False
ترک پان د ریپلی توسط ریانا خوانده و شرکت دف جم آن را منتشر کرده است
3
False
ترک پاپ مرده‌است توسط ردیوهد خوانده و کمپانی ای‌ام‌آی منتشر کرده
3
False
ترک پایان خوش من توسط اوریل لوین خوانده و کمپانی آریستا منتشر کرده است
3
False
ترک پایان زمان (ترانه) توسط بیانسه خوانده و کمپانی کلمبیا رکوردز آن را منتشر کرده است
3
False
ترک آسودگی (ترانه) توسط تیلور سوئیفت خوانده و بیگ مشین رکوردز منتشر کرده
3
False
ترک بخوان (ترانه مای کمیکال رومنس) توسط مای کمیکال رومنس خوانده و رپرایز رکوردز منتشر کرده

In [99]:
print(len(generatedRelations))

612


In [100]:
resultDataFile = open("generated-complex-new-pattern-10.json", "w", encoding='utf-8')
resultDataFile.write(str(json.dumps(generatedRelations, indent=4, sort_keys=True, ensure_ascii=False)))
resultDataFile.close()